This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

In [71]:
%pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
%pip install selenium webdriver-manager python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [72]:
import os
import boto3

In [73]:
# The file download processing section should be placed here

#...........................................................

In [74]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'DO NOT SHOW KEY HERE')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'DO NOT SHOW KEY HERE')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'DO NOT SHOW KEY HERE')

data_folder_path = "data/data-raw"

In [75]:
""" 4. Use Selenium to download file from Google Drive """

from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import time

# If data.zip exists, delete it
if os.path.exists(data_folder_path):
    for f in os.listdir(data_folder_path):
        if f == "data.zip":
            os.remove(f'{data_folder_path}/data.zip')

service = Service(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()
# Need to start fullscreen so the download button would be visible and clickable
chrome_options.add_argument("--start-fullscreen")

# Set default download directory to current project directory
current_dir = os.getcwd()
chrome_options.add_experimental_option("prefs", {"download.default_directory": f'{current_dir}/{data_folder_path}'})

browser = webdriver.Chrome(service=service, options=chrome_options)

try:
    browser.get("https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC?usp=sharing")

    time.sleep(5)

    # Find the file row
    file_link = browser.find_element(By.XPATH, f'//div[.="data.zip"]/ancestor::div[@role="gridcell"]')

    # Hover the row so the download button would be visible and clickable
    action = ActionChains(browser)
    action.move_to_element(file_link).perform()

    # Click the download button
    # Class name might not be the best idea, could use another one
    # And using aria-label should be considered since it could be `Download` or `Tải xuống` based on region
    download = browser.find_element(By.CLASS_NAME, "akerZd")
    download.click()

    # Wait for downloading
    time.sleep(10)
except Exception as e:
    raise e
finally:
    browser.quit()

In [76]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    try:
        s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)
    except Exception as e:
        raise e


In [77]:
""" 5. Unzip file and upload to MinIO """

from zipfile import ZipFile

with ZipFile(f'{data_folder_path}/data.zip', "r") as zip:
    zip.extractall(data_folder_path)

In [78]:
# Upload file to minio
upload_file_to_minio(f'{data_folder_path}/data.json', 'data', 'data-raw/data.json')
upload_file_to_minio(f'{data_folder_path}/data2.json', 'data', 'data-raw/data2.json')
upload_file_to_minio(f'{data_folder_path}/data3.json', 'data', 'data-raw/data3.json')